# 🚀 BREADBOARD AI - GOLDEN LAUNCHER
**Verified Fixes:**
- ✅ **Deep Clean**: Wipes `/content/app` to prevent stale file conflicts.
- ✅ **Mermaid Fix**: Explicitly installs `mermaid` dependency.
- ✅ **Config Sync**: Includes `ui`, `index.html` and all config files.
- ✅ **Explicit Launch**: Uses `npx vite ui` to match `root: 'ui'` in config.
- ✅ **Blue Link**: Uses `eval_js` for reliable proxy URL generation.

In [ ]:
# 1. INITIALIZE & MOUNT
import os, shutil
from google.colab import drive

print("[1/4] Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
except: pass

APP_ROOT = '/content/app'
DRIVE_ROOT = '/content/drive/MyDrive/CodingProjects/BreadboardAI'

print("[2/4] Checking workspace...")
if not os.path.exists(APP_ROOT):
    os.makedirs(APP_ROOT)
    print("[INFO] Created fresh workspace.")
else:
    print("[INFO] Workspace already exists. Skipping deep clean.")
# Pro Tip: To force a deep clean, run: !rm -rf /content/app

In [ ]:
# 2. INSTALL DEPENDENCIES
import os, shutil, subprocess
os.chdir(APP_ROOT)

print("[3/4] Syncing package.json...")
shutil.copy2(os.path.join(DRIVE_ROOT, 'package.json'), APP_ROOT)

# Only install if node_modules is missing
if not os.path.exists('node_modules'):
    print("[TASK] Installing dependencies (First time setup - may take 2 mins)... ")
    subprocess.run(['npm', 'install', '--no-audit', '--no-fund', '--silent'], check=True)
    print("[TASK] Patching missing dependencies (Mermaid)...")
    subprocess.run(['npm', 'install', 'mermaid', '--save', '--no-audit', '--no-fund', '--silent'], check=True)
    print("[SUCCESS] Dependencies installed.")
else:
    print("[INFO] node_modules found. Skipping installation.")

In [ ]:
# 3. SYNC FILES
import os, shutil
os.chdir(APP_ROOT)

print("[TASK] Syncing project source (UI, Src, Configs)...")
# Explicitly listing ALL required files/folders
sync_targets = ['src', 'public', 'ui', 'vite.config.ts', 'tsconfig.json', '.env', 'index.html']

for item in sync_targets:
    src_path = os.path.join(DRIVE_ROOT, item)
    dst_path = os.path.join(APP_ROOT, item)
    
    if os.path.exists(src_path):
        if os.path.isdir(src_path):
            shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
        else:
            shutil.copy2(src_path, dst_path)
    else:
        print(f"[WARNING] Skipped missing item: {item}")

print("[SUCCESS] Files synced.")

In [ ]:
# 4. LAUNCH & LINK
import time
from google.colab.output import eval_js
os.chdir(APP_ROOT)

print("[TASK] Killing old processes...")
get_ipython().system_raw('fuser -k 5173/tcp')
get_ipython().system_raw('fuser -k 3000/tcp')

print("[TASK] Starting Express API (Port 3000)...")
get_ipython().system_raw('npx tsx src/bridge/server.ts > server_log.txt 2>&1 &')

print("[TASK] Starting Vite (Port 5173, Root: ui)...")
# CRITICAL FIX: explicitly pointing to 'ui' folder and config
get_ipython().system_raw('npx vite ui --config vite.config.ts --port 5173 --host 0.0.0.0 > vite_log.txt 2>&1 &')

print("[WAIT] Warming up (15s)...")
time.sleep(15)

print("Generating access link...")
try:
    url = eval_js("google.colab.kernel.proxyPort(5173)")
    print(f"\n{'='*50}\n 🔗 CLICK HERE: {url} \n{'='*50}")
except Exception as e:
    print(f"[ERROR] Link generation failed: {e}")
    
# Tail logs in case of error
print("\n--- Recent Logs ---")
get_ipython().system('tail -n 5 vite_log.txt')

In [ ]:
# 5. ENABLE LIVE AUTO-SAVE (Optional)
# This creates a safe symlink from /content/app/output -> Drive
# Any files written to 'output' will be instantly saved to Drive.
import sys
if APP_ROOT not in sys.path:
    sys.path.append(APP_ROOT)

try:
    from src.utils.setup_autosave import setup_autosave
    print("[TASK] Configuring Live Auto-Save...")
    setup_autosave()
    print("[SUCCESS] Auto-Save is ACTIVE. Files in 'output' are safe on Drive.")
except ImportError:
    print("[ERROR] Could not import setup_autosave. Ensure Step 3 (Sync) ran successfully.")